In [3]:
import numpy as np

a = np.array([1, 2, 3])
b = np.array([4, 5, 6])

np.maximum(a, b)

array([4, 5, 6])

In [7]:
from main_mfrl_REINFORCE_RA2C import Pinet as ra2c
from main_mfrl_REINFORCE_recurrent import Pinet as recurrent
from main_mfrl_REINFORCE_vanilla import Pinet as vanilla
from mfrl_lib.lib import *

import torch

# if GPU is to be used
if torch.cuda.is_available():
    device = torch.device("cuda")
elif torch.backends.mps.is_available():
    device = torch.device("mps")
else:
    device = torch.device("cpu")

# Assuming you've defined these constants elsewhere or importing them too
N_OBSERVATIONS = 3
N_ACTIONS = 2

# Create an instance of Pinet
models = [ra2c(N_OBSERVATIONS, N_ACTIONS).to(device) for _ in range(8)]
for i in range(8):
    models[i].load_state_dict(torch.load(f"models/RA2C_agent_{i}_20240816_113309.pth", map_location=device))
    models[i].eval()



In [8]:
# Use the model
observation = torch.randn(1, 1, N_OBSERVATIONS).to(device)  # Example input
h = torch.zeros(1, 1, 32).to(device)  # Initial hidden state
c = torch.zeros(1, 1, 32).to(device)  # Initial cell state

for i in range(8):
    model = models[i]
    action, log_prob, entropy, value, new_h, new_c = model.sample_action(observation, h, c)
    print(f"Sampled action for agent {i}: {action.item()}")
    print(f"Estimated value for agent {i}: {value.item()}")

Sampled action for agent 0: 0
Estimated value for agent 0: 0.47969698905944824
Sampled action for agent 1: 0
Estimated value for agent 1: -0.6916403770446777
Sampled action for agent 2: 0
Estimated value for agent 2: -0.2694806456565857
Sampled action for agent 3: 0
Estimated value for agent 3: -0.29422807693481445
Sampled action for agent 4: 0
Estimated value for agent 4: -0.34012719988822937
Sampled action for agent 5: 1
Estimated value for agent 5: -0.5117686986923218
Sampled action for agent 6: 1
Estimated value for agent 6: -0.6181781888008118
Sampled action for agent 7: 0
Estimated value for agent 7: 0.49812790751457214
